In [ ]:
!unzip /content/drive/MyDrive/CatSound_DataSet_V2.zip -d /content/drive/MyDrive/Cat

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Cat/NAYA_DATA_AUG1X/Defence/Edit9062Hiss_aug1(1).mp3  
  inflating: /content/drive/MyDrive/Cat/NAYA_DATA_AUG1X/Defence/Edit9063Hiss.mp3  
  inflating: /content/drive/MyDrive/Cat/NAYA_DATA_AUG1X/Defence/Edit9063Hiss_aug1(1).mp3  
  inflating: /content/drive/MyDrive/Cat/NAYA_DATA_AUG1X/Defence/Edit9064Hiss.mp3  
  inflating: /content/drive/MyDrive/Cat/NAYA_DATA_AUG1X/Defence/Edit9064Hiss_aug1(1).mp3  
  inflating: /content/drive/MyDrive/Cat/NAYA_DATA_AUG1X/Defence/Edit9065Hiss.mp3  
  inflating: /content/drive/MyDrive/Cat/NAYA_DATA_AUG1X/Defence/Edit9065Hiss_aug1(1).mp3  
  inflating: /content/drive/MyDrive/Cat/NAYA_DATA_AUG1X/Defence/Edit9066Hiss.mp3  
  inflating: /content/drive/MyDrive/Cat/NAYA_DATA_AUG1X/Defence/Edit9066Hiss_aug1(1).mp3  
  inflating: /content/drive/MyDrive/Cat/NAYA_DATA_AUG1X/Defence/Edit9067Hiss.mp3  
  inflating: /content/drive/MyDrive/Cat/NAYA_DATA_AUG1X/Defence/Edit9067Hiss_au

In [ ]:
!pip install pydub
!apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import os
from pydub import AudioSegment

def convert_mp3_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio.export(wav_path, format="wav")

def create_datasets(input_dir, output_dir_aug, output_dir_other):
    for class_dir in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_dir)

        if os.path.isdir(class_path):
            class_dir_aug = os.path.join(output_dir_aug, class_dir)
            class_dir_other = os.path.join(output_dir_other, class_dir)
            os.makedirs(class_dir_aug, exist_ok=True)
            os.makedirs(class_dir_other, exist_ok=True)

            for filename in os.listdir(class_path):
                if filename.endswith(".mp3"):
                    file_path = os.path.join(class_path, filename)
                    wav_filename = filename.replace(".mp3", ".wav")

                    if "aug" in filename:
                        output_wav_path = os.path.join(class_dir_aug, wav_filename)
                    else:
                        output_wav_path = os.path.join(class_dir_other, wav_filename)

                    convert_mp3_to_wav(file_path, output_wav_path)
                    print(f"Converted {filename} to {output_wav_path}")

input_directory = '/content/drive/MyDrive/Cat/NAYA_DATA_AUG1X'
output_directory_aug = '/content/drive/MyDrive/Cat/raw'
output_directory_other = '/content/drive/MyDrive/Cat/aug'

create_datasets(input_directory, output_directory_aug, output_directory_other)


Streaming output truncated to the last 5000 lines.
Converted Edit9056Hiss.mp3 to /content/drive/MyDrive/Cat/aug/Defence/Edit9056Hiss.wav
Converted Edit9056Hiss_aug1(1).mp3 to /content/drive/MyDrive/Cat/raw/Defence/Edit9056Hiss_aug1(1).wav
Converted Edit9057Hiss.mp3 to /content/drive/MyDrive/Cat/aug/Defence/Edit9057Hiss.wav
Converted Edit9057Hiss_aug1(1).mp3 to /content/drive/MyDrive/Cat/raw/Defence/Edit9057Hiss_aug1(1).wav
Converted Edit9058Hiss.mp3 to /content/drive/MyDrive/Cat/aug/Defence/Edit9058Hiss.wav
Converted Edit9058Hiss_aug1(1).mp3 to /content/drive/MyDrive/Cat/raw/Defence/Edit9058Hiss_aug1(1).wav
Converted Edit9060Hiss.mp3 to /content/drive/MyDrive/Cat/aug/Defence/Edit9060Hiss.wav
Converted Edit9060Hiss_aug1(1).mp3 to /content/drive/MyDrive/Cat/raw/Defence/Edit9060Hiss_aug1(1).wav
Converted Edit9061Hiss.mp3 to /content/drive/MyDrive/Cat/aug/Defence/Edit9061Hiss.wav
Converted Edit9061Hiss_aug1(1).mp3 to /content/drive/MyDrive/Cat/raw/Defence/Edit9061Hiss_aug1(1).wav
Converted

In [ ]:
import os
import random
from shutil import move

def split_data(file_list, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    random.shuffle(file_list)
    total_files = len(file_list)
    train_split = int(total_files * train_ratio)
    val_split = int(total_files * (train_ratio + val_ratio))
    return file_list[:train_split], file_list[train_split:val_split], file_list[val_split:]

def process_class_files(class_path):
    all_files = []
    for filename in os.listdir(class_path):
        if filename.endswith(".wav"):
            file_path = os.path.join(class_path, filename)
            all_files.append(file_path)
    return all_files
def move_files(file_list, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for file_path in file_list:
        dest_path = os.path.join(output_dir, os.path.basename(file_path))
        move(file_path, dest_path)

def create_datasets(input_dir, output_base_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    for class_dir in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_dir)

        if os.path.isdir(class_path):
            all_files = process_class_files(class_path)

            train_files, val_files, test_files = split_data(all_files, train_ratio, val_ratio, test_ratio)

            train_dir = os.path.join(output_base_dir, 'train', class_dir)
            val_dir = os.path.join(output_base_dir, 'val', class_dir)
            test_dir = os.path.join(output_base_dir, 'test', class_dir)

            move_files(train_files, train_dir)
            move_files(val_files, val_dir)
            move_files(test_files, test_dir)

input_directory = '/content/drive/MyDrive/Cat/raw'
output_base_directory = '/content/drive/MyDrive/Cat/Cleaned/raw'

create_datasets(input_directory, output_base_directory)
